In [2]:
import pandas as pd 
import numpy as np 
import os

In [11]:
def data_generator():
    
    directories = os.listdir('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')
    directories = directories[:2]#temporal, indica cuantos batches se van a tomar
    directories = [('/kaggle/input/icecube-neutrinos-in-deep-ice/train/')+x for i, x in enumerate(directories)]
    
    data = pd.DataFrame()
    
    
    for file in directories:
        temp = pd.read_parquet(file)
        
        temp = temp.sort_values('charge',ascending = False).groupby('event_id').head(2)
        # al poner el head en 100 se reduciria la tabla en un 98%
    
        data = pd.concat([data, temp], axis=0)
        
    
    #data = data.drop_duplicates()##
    
    coords = pd.read_csv('/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
    train_meta = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/train_meta.parquet')
    
    data['event'] = data.index
    
    
    data = pd.merge(data, train_meta[['event_id', 'azimuth', 'zenith']], left_on=data['event'], right_on=train_meta['event_id'].astype(float))
    
    data = pd.merge(data, coords, on='sensor_id', how='outer')


     
    data = pd.concat([
        data.groupby('event').agg({'time': lambda x: list(x)}),
        data.groupby('event').agg({'charge': lambda x: list(x)}),
        data.groupby('event').agg({'x': lambda x: list(x)}),
        data.groupby('event').agg({'y': lambda x: list(x)}),
        data.groupby('event').agg({'z': lambda x: list(x)}),
        data.groupby('event')['azimuth'].apply(min),
        data.groupby('event')['zenith'].apply(min)
        ],axis=1)
    
    
    return data[['time','charge','x','y','z','azimuth','zenith']]

In [12]:
data = data_generator()

In [13]:
data.head(20)

,time,charge,x,y,z,azimuth,zenith
event,,,,,,,
777938857.0,"[11127.0, 10436.0]","[2.2750000953674316, 2.174999952316284]","[-32.96, 303.41]","[62.44, 335.64]","[19.23, 70.42]",0.095967,2.747172
777938862.0,"[10404.0, 10855.0]","[2.674999952316284, 1.875]","[11.87, 505.27]","[179.19, 257.88]","[-365.71, -140.55]",2.533461,0.638181
777938866.0,"[11231.0, 11027.0]","[2.075000047683716, 1.975000023841858]","[-200.55, -166.4]","[-74.03, -287.79]","[-225.49, -421.34]",2.510193,1.059650
777938893.0,"[15461.0, 6528.0]","[1.774999976158142, 1.774999976158142]","[-200.55, 237.78]","[-74.03, -442.42]","[-208.47, 382.32]",4.844315,1.450211
777938910.0,"[6721.0, 10971.0]","[1.9249999523162842, 2.424999952316284]","[132.03, 371.56]","[202.98, -92.18]","[94.24, 399.46]",4.235402,1.387627
777938957.0,"[9773.0, 9905.0]","[4.224999904632568, 2.375]","[106.94, -245.65]","[27.09, -190.49]","[-230.85, 467.44]",4.699791,0.707415
777938967.0,"[10283.0, 10724.0]","[2.7750000953674316, 2.2249999046325684]","[224.58, -200.55]","[432.35, -74.03]","[173.67, 472.36]",6.216608,2.202253
777938986.0,"[14845.0, 9902.0]","[1.975000023841858, 2.0250000953674316]","[-111.51, 224.58]","[159.98, 432.35]","[432.45, -490.14]",3.402021,1.519049
777938998.0,"[15742.0, 10577.0]","[2.125, 1.5750000476837158]","[-570.9, 72.37]","[-125.14, -66.6]","[465.56, -426.16]",5.829281,0.347467


In [ ]:
size = 0.2
test_size = round((len(data)*size),0)

data.shuffle()

X = data[['time','charge','x','y','z']]
y = data[['azimuth','zenith']]

X_train = X.iloc[test_size:]
X_test = X.iloc[:test_size]

y_train = y.iloc[test_size:]
y_test = y.iloc[:test_size]


In [ ]:
input = keras.Input(shape=(7,100))

x = layers.Flatten()(input)
x = layers.Dense(700, activation='linear')(x)
x = layers.Dense(350, activation='linear')(x)
x = layers.Dense(70, activation='linear')(x)

out_az = keras.layers.Dense(1, activation='relu', name='az-out')(x)
out_ze = keras.layers.Dense(1, activation='relu', name='ze-out')(x)


encoder = keras.Model( inputs = input, outputs = [out_az, out_ez], name='encoder')


encoder.compile(
    loss = {
        'az-out': tf.keras.losses.MeanAbsoluteError(),
        'ze-out': tf.keras.losses.MeanAbsoluteError(),
    },

    metrics = {
        'az-out': 'accuracy',
        'ze-out': 'accuracy',
    },

    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    

encoder.fit(xtrain, [y_az, y_ze], epochs=10, verbose=2)
    

In [9]:
sample #submission

,event_id,azimuth,zenith
0,2092,1,1
1,7344,1,1
2,9482,1,1
